# <center>L2 Computational Physics</center>
---

## Week 5: Monte Carlo Methods

In this notebook, you will simulate a system with of three nuclei $A$, $B$ and $C$ where $A$ decays into $B$ and $B$ decays into $C$. If exposed to a neutron flux nucleus $C$ can be activated into a nucleus $A$. 

In [3]:
import numpy
from matplotlib import pyplot as plt
import random

Implement a function that tells whether a transition has occured, based on the transition probability and a random number. Use the random number `r` from `random.random()` and use the procedure described in the notes so that the checks can work in a reproducible way.

In [4]:
def has_transitioned(prob):
    r = random.random()
    # YOUR CODE HERE
    if r<prob:
        return True
    else:
        return False

In [5]:
# this test is worth 1 mark
random.seed(9867)
assert [ has_transitioned(0.5) for i in range(10)] == [False, False, True, False, False, False, False, True, False, True]

We are going to keep track of the state of the atoms using a either `'A'`, `'B'` or `'C'` for each nucleus. For example 
```python
state = ['A', 'A', 'B', 'A', 'C']
```

We will capture the possible physical reaction in a list of allowed transitions such as

`('A', 'B', 0.25 )`

where the first element is the initial state, the second element is the final state of the transition and its probability in the time interval considered is the third argument.

Define a function that takes as arguments the current state and a list of such transition rules and implements the transition (or not) and returns the new state

In [6]:
def evolveOne(currentState, rules):
    # YOUR CODE HERE
    states1=[i[0]for i in rules]
    states2=[i[1]for i in rules]
    probs=[i[2]for i in rules]
    if currentState in states1:
        n=states1.index(currentState)
    else:
        return currentState
        end()
    if has_transitioned(probs[n]) is True:
        return states2[n]
    else:
        return currentState
    
    


In [7]:
# these tests are worth 1 mark
alwaysDecayRules = [
    ('A', 'B', 1.0),
    ('B', 'C', 1.0)
]
assert evolveOne('A', alwaysDecayRules) == 'B'
assert evolveOne('B', alwaysDecayRules) == 'C'

In [8]:
# these tests are worth 2 mark
random.seed(112211)
testRules = [
    ('A', 'B', 0.33),
    ('B', 'C', 0.75)
]
assert evolveOne('A', testRules) == 'A'
assert evolveOne('A', testRules) == 'A'
assert evolveOne('A', testRules) == 'A'
assert evolveOne('A', testRules) == 'A'
assert evolveOne('A', testRules) == 'B'

assert evolveOne('B', testRules) == 'B'
assert evolveOne('B', testRules) == 'C'
assert evolveOne('B', testRules) == 'C'
assert evolveOne('B', testRules) == 'C'
assert evolveOne('B', testRules) == 'C'

# with no rules there should be no change
assert evolveOne('C', testRules) == 'C'


Now implement a function that takes a list of states and transition them according to the rules passed as argument. This function should return a new vector of states, it should not modify the state passed as an argument!

In [9]:
def evolveMany(states, rules):
    newState = []
    # YOUR CODE HERE
    for item in states:
        newState.append(evolveOne(item, rules))
    return newState


In [10]:
# these tests are worth 1 mark
random.seed(112287)
testRules = [
    ('A', 'B', 0.33),
    ('B', 'C', 0.75)
]
initialTestArray = ['A','B','C']*5
evolvedTest = evolveMany(initialTestArray, testRules)
targetArray = ['B', 'C', 'C', 'A', 'C', 'C', 'A', 'B', 'C', 'A', 'C', 'C', 'B', 'C', 'C'] 
assert evolvedTest == targetArray
# checks the initial array is left unchanged
assert initialTestArray == ['A','B','C']*5

Define a function that evolves a system that starts with initial amounts `NA`, `NB` and `NC` of $A$, $B$ and $C$ nuclei and evolved it in `n_timestep` from time $t=0$ to $t=t_{max}$. The function should return three arrays, one for each atom type, of the number of nuclei of that type at each time step. Each array should contain `n_timestep+1` elements including the initial amount. 
 

In [11]:
def evolve_system(NA, NB, NC, rules, n_step):
    state = (['A'] * NA)+(['B'] * NB)+(['C'] * NC)

    A_count = numpy.empty(n_step + 1, dtype=int)
    B_count = numpy.empty(n_step + 1, dtype=int)
    C_count = numpy.empty(n_step + 1, dtype=int)
    nuclei = []
    # YOUR CODE HERE
    nuclei.extend ('A' for i in range(NA))
    nuclei.extend('B' for i in range(NB))
    nuclei.extend('C' for i in range(NC))
    for i in range(n_step+1):
        A_count[i]=nuclei.count('A')
        B_count[i]=nuclei.count('B')
        C_count[i]=nuclei.count('C')
        nuclei= evolveMany(nuclei, rules)
    return A_count, B_count, C_count

In [12]:
# these tests are worth 2 marks
rules = [
    ('A', 'B', 0.0033),
    ('B', 'C', 0.0075),
    ('C', 'A', 0.009)
    
]

r1, r2, r3 = evolve_system(0,0,250, rules, 17)
assert len(r1) == 18
assert len(r2) == 18
assert len(r3) == 18

In [13]:
# these tests are worth 2 marks 
testrules = [
    ('A', 'B', 0.086),
    ('B', 'C', 0.075),
    ('C', 'A', 0.19)
    
]

random.seed(9485)
r1, r2, r3 = evolve_system(200,200,200, testrules, 20)
assert (r1 == [200, 213, 233, 250, 258, 251, 266, 263, 259, 260, 265, 259, 256,
        255, 258, 256, 259, 253, 249, 247, 253]).all()
assert (r2 == [200, 198, 201, 206, 205, 214, 214, 212, 216, 221, 225, 234, 236,
        238, 234, 235, 231, 245, 253, 256, 252]).all()
assert (r3 == [200, 189, 166, 144, 137, 135, 120, 125, 125, 119, 110, 107, 108,
        107, 108, 109, 110, 102,  98,  97,  95]).all()

## Plotting tasks

Create a plot with the number of $A$, $B$ and $C$ nuclei, starting with 250 $C$ nuclei and evolving the system for 100 hours using 200 steps and with neutron flux on. Evolve the system for another 100 hours (using 200 steps) without neutron flux (i.e. no transitions from $C$ to $A$).  

The half life of the $A$ atoms is 10.1 hours, the half life of $B$ nuclei is 15.7 hours and we can caracterise the rate of activation of $C$ into $A$ when the neutron flux is on with and effective half-life of 3.2 hours.

The plot should have the appropriate labels and legend. [8 marks]    




In [14]:
steps = 200
t_total = 100
t_half_A = 10.1
t_half_B = 15.7
t_half_C = 3.2
rules1 = [
    ('A', 'B', (numpy.log(2))/t_half_A),
    ('B', 'C', (numpy.log(2))/t_half_B),
    ('C', 'A', (numpy.log(2))/t_half_C)   
]
rules2 = [
    ('A', 'B', (numpy.log(2))/t_half_A),
    ('B', 'C', (numpy.log(2))/t_half_B)  
]


NFon=evolve_system(0, 0, 250, rules1, nsteps)
NFoff=evolve_system(NFon[0][200],NFon[1][200],NFon[2][200],rules2,nsteps)
A=numpy.append(NFon[0][:],NFoff[0][:])
B=numpy.append(NFon[1][:],NFoff[1][:])
C=numpy.append(NFon[2][:],NFoff[2][:])

system(nsteps,NFonRules,NFoffRules)
time=[]
time.extend(0.5*n for n in range(2*steps+2))

plt.plot(time, A, label='A nuclei')
plt.plot(time, B, label='B nuclei')
plt.plot(time, C, label='C nuclei')
plt.ylabel('Number of Nuclei')
plt.xlabel('Time (hours)')
plt.title('The Decay of Various Nuclei Over Time')
plt.legend()

NameError: name 'system' is not defined

Run the above simulation 20 times with 200 steps and use the results to calculate an average and the uncertainty on the number of $A$ atoms as a function of time. Use and `errorbar` plot for it. You might be interested in the `numpy.average` and `numpy.std` functions. The plot should have axis labels and a title.  [3 marks]


In [15]:
nsim = 20
nsteps = 200
t_total = 100
t_half_A = 10.1
t_half_B = 15.7
t_half_C = 3.2
rules1 = [
    ('A', 'B', (numpy.log(2))/t_half_A),
    ('B', 'C', (numpy.log(2))/t_half_B),
    ('C', 'A', (numpy.log(2))/t_half_C)   
]
rules2 = [
    ('A', 'B', (numpy.log(2))/t_half_A),
    ('B', 'C', (numpy.log(2))/t_half_B)  
]
As=[]

for i in range (nsim):
    NFon=evolve_system(0, 0, 250, rules1, nsteps)
    NFoff=evolve_system(NFon[0][200],NFon[1][200],NFon[2][200],rules2,nsteps)
    A=numpy.append(NFon[0][:],NFoff[0][:])
    As.append(A)
print (As)
#allas=numpy.asarray(As)
#stdev = numpy.std(allas, axis=0)
#ave = numpy.average(allas, axis=0)
#error = (1/numpy.sqrt(nsim))*stdev
#time=[]
#time.extend(0.5*n for n in range(2*steps+2))

#plt.errorbar(time,ave,yerr=error)
#plt.ylabel('Average Number of Nuclei')
#plt.xlabel('Time (hours)')
#plt.title('How the Number of A Nuclei Changes Over Time')

[array([  0,  53,  94, 124, 135, 145, 167, 167, 172, 170, 173, 166, 158,
       145, 133, 130, 123, 121, 122, 119, 118, 115, 108, 108, 106,  99,
        98,  89,  84,  85,  84,  86,  90,  92,  96,  92,  88,  92,  87,
        87,  88,  95,  93,  93,  87,  87,  90,  92,  89,  90,  93,  96,
        92,  87,  81,  82,  77,  79,  73,  75,  76,  77,  77,  77,  79,
        82,  77,  78,  81,  84,  81,  85,  89,  87,  82,  76,  77,  75,
        74,  78,  83,  82,  81,  80,  81,  88,  92,  93,  96,  90,  84,
        88,  86,  78,  78,  80,  82,  86,  89,  86,  84,  88,  87,  92,
        92,  88,  93,  90,  87,  91,  90,  92,  91,  84,  86,  89,  97,
        90,  88,  81,  83,  84,  85,  78,  78,  74,  69,  67,  67,  67,
        70,  72,  78,  79,  82,  87,  89,  87,  85,  85,  86,  90,  85,
        84,  90,  95,  97,  93,  95,  92,  91,  89,  88,  85,  89,  98,
        97,  98, 102,  99, 102, 102, 105,  97,  97,  95,  91,  88,  84,
        88,  81,  85,  83,  84,  87,  89,  88,  81,  86,  80,  